### Import libs and dependencies

In [ ]:
import google.generativeai as genai
import json
import pandas as pd
import time
import os
import re
from collections import Counter
from tqdm import tqdm
from dotenv import load_dotenv

## Setup

### Settings Gemini API and Model

In [ ]:
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

genai.configure(api_key=gemini_api_key)

model = genai.GenerativeModel("gemini-2.5-flash")

### Safety Settings

In [ ]:
# Phòng khi tên thuốc có thể vô tính dính phải blacklist từ cấm, như morphine, ketamine, ...
SAFETY_SETTINGS = [
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"}
        ]

### Prompt để trích xuất thực thể và quan hệ

In [114]:
SYSTEM_PROMPT = """
Bạn là một chuyên gia về Dược điển Việt Nam, xử lý ngôn ngữ tự nhiên và trích xuất tri thức Y Dược cho đồ thị Neo4j. 
Tôi sẽ cung cấp cho bạn một danh sách các đoạn văn bản (Chunks), mỗi đoạn có một ID riêng.

NHIỆM VỤ:
Trích xuất các Thực thể (Entities) và Quan hệ (Relations) trong từng đoạn văn bản và phân loại chúng theo đúng định nghĩa sau:
    
1. DANH SÁCH LOẠI THỰC THỂ CẦN TÌM:
- DRUG : Tên Thuốc, chế phẩm. Ví dụ: Huyết thanh kháng bạch hầu, Interferon Alpha 2, Tuberculin PPD.
- CHEMICAL: Hóa chất, hoạt chất. Ví dụ: Protein, Globulin miễn dịch, Amoni sulfat, Phenol.
- DISEASE: Tên bệnh. Ví dụ: Viêm gan B, Bạch hầu, Lao, Dại, Uốn ván.
- ORGANISM: Tên vi sinh vật. Ví dụ: Mycobacterium tuberculosis, Virus viêm gan B, VSV, MDBK.
- TEST_METHOD: Phương pháp kiểm nghiệm. Ví dụ: Điện di miễn dịch, ELISA, Nuôi cấy tế bào, Phổ hấp thụ.
- STANDARD: Tiêu chuẩn chất lượng. Ví dụ: pH: 6.4-7.2, Nhiệt độ: 2-8°C, Hiệu giá: ≥100 IU/mL
- STORAGE_CONDITION: Điều kiện bảo quản. Ví dụ: Lạnh: 2-10°C, Tránh ánh sáng, Không làm đông băng.
- PRODUCTION_METHOD: Phương pháp sản xuất. Ví dụ: Tái tổ hợp ADN, Nuôi cấy tế bào, Tách chiết protein.

2. DANH SÁCH QUAN HỆ CẦN TÌM:
- CONTAINS: Có dạng DRUG -> CHEMICAL (Thuốc chứa thành phần)
- TREATS: Có dạng DRUG -> DISEASE (Thuốc điều trị bệnh)
- TARGETS: Có dạng DRUG -> ORGANISM (Thuốc tác động lên vi sinh vật)
- TESTED_BY: Có dạng DRUG -> TEST_METHOD (Thuốc được kiểm nghiệm bằng)
- HAS_STANDARD: Có dạng DRUG -> STANDARD (Thuốc có tiêu chuẩn kỹ thuật)
- STORED_AT: Có dạng DRUG -> STORAGE_CONDITION (Thuốc được bảo quản ở điều kiện)
- PRODUCED_BY: Có dạng DRUG -> PRODUCTION_METHOD (Thuốc được sản xuất bằng phương pháp)
- REQUIRES: Có dạng TEST_METHOD -> CHEMICAL (Phương pháp kiểm nghiệm yêu cầu hóa chất)

3. YÊU CẦU ĐẦU RA (JSON FORMAT):
Trả về một Mảng (List) các object, mỗi phần tử ứng với một Chunk đầu vào:
[
  {
    "chunk_id": (Giữ nguyên ID kiểu số hoặc chuỗi như đầu vào),
    "relations": [
      {
        "entity1_text": "tên của entity 1",
        "entity1_type": "loại của entity 1",
        "entity2_text": "tên của entity 2",
        "entity2_type": "loại của entity 2",
        "relationship_type": "loại quan hệ",
        "context_text": "Copy nguyên văn câu, hoặc copy phần văn bản trong chunk có chứa mối quan hệ này."
      }
    ]
  }
]

QUY TẮC QUAN TRỌNG:
- OUTPUT BẮT BUỘC LÀ JSON THUẦN (Không Markdown, không kí tự thừa thãi).
- Nếu chunk không có quan hệ nào, trả về danh sách "relations" rỗng.
- Chỉ xét các quan hệ giữa DRUG và các thực thể khác, hoặc giữa TEST_METHOD và CHEMICAL, tuyệt đối không xét quan hệ giữa các thực thể khác.
- Các thuật ngữ Entity phải copy chính xác 100% từ văn bản gốc để tôi có thể tìm vị trí index sau này.
- Chỉ trích xuất thông tin có trong văn bản, không được bịa đặt, không tự ý dịch thuật ngữ.
"""

### Config

In [ ]:
# Đường dẫn file
INPUT_FILE = "../data/processed/merged_text.txt"
OUTPUT_FILE = "../data/processed/entities_relations.json"
MODEL_NAME = "gemini-2.5-flash" 
BATCH_SIZE = 10  # Số đoạn văn bản xử lý trong một lần gọi API
GENERATION_CONFIG = {
  "temperature": 0.1, # Temp range: [0.0, 2.0], Temp thấp để tăng accuracy
  "top_k": 64, # Top-K thấp để tăng accuracy
  "response_mime_type": "application/json",
}

model = genai.GenerativeModel(
  model_name=MODEL_NAME,
  safety_settings=SAFETY_SETTINGS,
  generation_config=GENERATION_CONFIG
)

### Batch prompt to attach chunks to the system prompt

In [116]:
def create_batch_prompt(batch_chunks):
    """Tạo nội dung prompt cho một batch"""
    prompt_text = "\n\nDƯỚI ĐÂY LÀ CÁC CHUNK CẦN XỬ LÝ:\n"
    for item in batch_chunks:
        prompt_text += f"ID: {item['chunk_id']}\nCONTENT: {item['text']}\n"
    return prompt_text

### Functions

In [ ]:
def load_data(filepath):
    '''Đọc file văn bản và tách thành các đoạn (chunks)'''
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    # Tách file và gán ID
    raw_fragments = content.split('</break>')
    data_chunks = []
    for idx, text in enumerate(raw_fragments):
        text = text.strip()
        if len(text) > 20: # Bỏ qua đoạn quá ngắn
            data_chunks.append({
                "chunk_id": idx,
                "text": text
            })
    print(f"Tổng số đoạn văn bản cần xử lý: {len(data_chunks)}")
    print(f"Tổng số chunk ban đầu: {len(raw_fragments)}")
    return data_chunks


def find_char_indices(full_text, sub_text):
    """
    Tìm vị trí bắt đầu và kết thúc của sub_text trong full_text.
    Ưu tiên tìm kiếm chính xác.
    """
    if not sub_text or not full_text: return -1, -1
    
    start = full_text.find(sub_text)
    if start != -1:
        return start, start + len(sub_text)
        
    return -1, -1

In [ ]:
def process_all_data(input_file, output_file):
    '''Xử lý toàn bộ dữ liệu từ file đầu vào và lưu kết quả ra file đầu ra
    Cấu trúc JSON đầu ra cho mỗi chunk:
    {
        "chunk_id": "ID của chunk",
        "chunk_text": "văn bản gốc đầy đủ của chunk",
        "relations": [
            {
            "relation_id": "{chunk_id}_{relation_index}",
            "entity1": {
                "text": "tên thực thể 1",
                "type": "loại thực thể 1 ",
                "start_char": "vị trí bắt đầu",
                "end_char": "vị trí kết thúc"
            },
            "relationship_type": "tên mối quan hệ",
            "entity2": {
                "text": "tên thực thể 2",
                "type": "loại thực thể 2",
                "start_char": "vị trí bắt đầu",
                "end_char": "vị trí kết thúc"
            },
            "context_text": "phần đoạn văn chứa relationship"
            }
        ],
        "relation_count": "số lượng relations trong chunk"
    }    
    '''
    chunks = load_data(input_file)
    
    extract_results = []
    
    # Chia chunks thành các batch nhỏ
    for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc="Processing Batches"):
        batch = chunks[i : i + BATCH_SIZE]
        
        # Gọi API Gemini
        try:
            prompt = create_batch_prompt(batch)
            response = model.generate_content(SYSTEM_PROMPT + "\n" + prompt)
            batch_result = json.loads(response.text)
            
            # Mapping & Indexing để tra cứu nhanh text gốc từ ID
            chunk_map = {item['chunk_id']: item['text'] for item in batch}
            
            for item in batch_result:
                c_id = item.get('chunk_id')
                original_text = chunk_map.get(c_id)
                
                # Nếu LLM trả về ID không khớp, bỏ qua
                if original_text is None: continue

                processed_relations = []
                raw_relations = item.get('relations', [])
                
                for idx, rel in enumerate(raw_relations):
                    # Tính toán vị trí Index
                    e1_start, e1_end = find_char_indices(original_text, rel['entity1_text'])
                    e2_start, e2_end = find_char_indices(original_text, rel['entity2_text'])
                    
                    # Thêm nếu tìm thấy text trong văn bản gốc
                    if e1_start != -1 and e2_start != -1:
                        processed_relations.append({
                            "relation_id": f"{c_id}_{idx}",
                            "entity1": {
                                "text": rel['entity1_text'],
                                "type": rel['entity1_type'],
                                "start_char": e1_start,
                                "end_char": e1_end
                            },
                            "relationship_type": rel['relationship_type'],
                            "entity2": {
                                "text": rel['entity2_text'],
                                "type": rel['entity2_type'],
                                "start_char": e2_start,
                                "end_char": e2_end
                            },
                            "context_text": rel.get('context_text', ''),
                        })

                # Cấu trúc cuối cùng cho Chunk
                final_chunk_obj = {
                    "chunk_id": str(c_id),
                    "chunk_original_text": original_text,
                    "relations": processed_relations,
                    "relation_count": len(processed_relations)
                }
                extract_results.append(final_chunk_obj)
                
        except Exception as e:
            print(f"Error in batch {i}: {e}")
            # Rate limit safety
            time.sleep(2)

        # Rate limit safety
        time.sleep(2)

    # Lưu file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(extract_results, f, ensure_ascii=False, indent=2)
    
    print(f"Xong! File lưu tại: {output_file}")

In [ ]:
def analyze_extracted_data(file_path):
    '''Phân tích và thống kê dữ liệu đã trích xuất từ file JSON để kiểm tra chất lượng'''
    print(f"Đang đọc file: {file_path} ...")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy file với đường dẫn {file_path}")
        return

    # Khởi tạo bộ đếm
    relation_counter = Counter()
    entity_unique_set = set()
    entity_type_counter = Counter()

    total_chunks = len(data)
    total_relations_extracted = 0

    # Duyệt qua từng chunk
    for chunk in data:
        rels = chunk.get('relations', [])
        total_relations_extracted += len(rels)
        
        for r in rels:
            # Thống kê Quan hệ (Edges)
            r_type = r.get('relationship_type', 'UNKNOWN')
            relation_counter[r_type] += 1
            
            # Thống kê Thực thể (Nodes)
            # Lưu dạng tuple (text, type) để đảm bảo tính duy nhất
            e1 = r.get('entity1')
            e2 = r.get('entity2')
            
            if e1:
                entity_unique_set.add((e1['text'].strip().lower(), e1['type']))
            if e2:
                entity_unique_set.add((e2['text'].strip().lower(), e2['type']))

    # Đếm số lượng thực thể unique theo loại
    for text, label in entity_unique_set:
        entity_type_counter[label] += 1

    # In kết quả
    print(f"\nBÁO CÁO THỐNG KÊ DỮ LIỆU TRÍCH XUẤT\n")
    print("-"*40)
    print(f"Tổng số đoạn văn (Chunks): {total_chunks}")
    print(f"Tổng số quan hệ (Edges) : {total_relations_extracted}")
    print(f"Tổng số thực thể (Nodes) : {len(entity_unique_set)} (đã loại bỏ trùng lặp)")
    
    print("\n" + "-"*40 + "\n")
    print("PHÂN BỐ CÁC LOẠI QUAN HỆ (RELATIONSHIPS)")
    print("-"*40)
    if relation_counter:
        # Sắp xếp giảm dần
        for r_type, count in relation_counter.most_common():
            print(f"{r_type:<20} : {count}")
    else:
        print("(Không tìm thấy quan hệ nào)")

    print("\n" + "-"*40 + "\n")
    print("PHÂN BỐ CÁC LOẠI THỰC THỂ (ENTITIES - UNIQUE)")
    print("-"*40)
    if entity_type_counter:
        for e_type, count in entity_type_counter.most_common():
            print(f"{e_type:<20} : {count}")
    else:
        print("(Không tìm thấy thực thể nào)")
    print("-"*40)

## Run

In [120]:
process_all_data(INPUT_FILE, OUTPUT_FILE)

Tổng số đoạn văn bản cần xử lý: 3583
Tổng số chunk ban đầu: 3610


Processing Batches:   0%|          | 0/359 [00:00<?, ?it/s]

Processing Batches: 100%|██████████| 359/359 [3:40:19<00:00, 36.82s/it]  

Xong! File lưu tại: ../data/processed/entities_relations.json


In [121]:
analyze_extracted_data(OUTPUT_FILE)

Đang đọc file: ../data/processed/entities_relations.json ...

BÁO CÁO THỐNG KÊ DỮ LIỆU TRÍCH XUẤT

----------------------------------------
Tổng số đoạn văn (Chunks): 3583
Tổng số quan hệ (Edges) : 4707
Tổng số thực thể (Nodes) : 2767 (đã loại bỏ trùng lặp)

----------------------------------------

PHÂN BỐ CÁC LOẠI QUAN HỆ (RELATIONSHIPS)
----------------------------------------
REQUIRES             : 2119
TESTED_BY            : 812
HAS_STANDARD         : 617
PRODUCED_BY          : 449
CONTAINS             : 379
TREATS               : 165
STORED_AT            : 134
TARGETS              : 32

----------------------------------------

PHÂN BỐ CÁC LOẠI THỰC THỂ (ENTITIES - UNIQUE)
----------------------------------------
CHEMICAL             : 798
DRUG                 : 550
STANDARD             : 535
TEST_METHOD          : 384
PRODUCTION_METHOD    : 259
DISEASE              : 124
STORAGE_CONDITION    : 87
ORGANISM             : 30
----------------------------------------
